# Orbit determination example
This notebook does the following:
* Download an orbit first guess from SpaceTrack
* Download laser ranging data
* Feed the data to Orekit
* Estimate the orbit
* Propagate and compare the orbit to the first guess

First, some parameters need to be defined for the orbit determination:
* Satellite ID in NORAD and COSPAR format
* Spacecraft mass: important for the drag term
* Start date: date where to begin the orbit determination

In [1]:
# Parameters
noradId = 42829
cosparId = '1704205' 
mass = 20.0 # kg

from datetime import datetime
startDate = datetime(2018, 7, 14) # Beginning of the orbit determination

The following sets up accounts for SpaceTrack (for orbit data) and the EDC API (for laser ranging data).
* A SpaceTrack account is required, it can be created for free at: https://www.space-track.org/auth/createAccount
* An EDC account is required, it can be created for free at: https://edc.dgfi.tum.de/en/register/

In [5]:
# Space-Track
identity_st = '' # Email address for Space-Track
import getpass
print('Enter SpaceTrack password for account {}'.format(identity_st))
password_st = getpass.getpass()
import spacetrack.operators as op
from spacetrack import SpaceTrackClient
st = SpaceTrackClient(identity=identity_st, password=password_st)

# EDC API
username_edc = '' # Username for EDC API
print('Enter EDC API password for account {}'.format(username_edc))
password_edc = getpass.getpass() # You will get prompted for your password
url = 'https://edc.dgfi.tum.de/api/v1/'

Enter SpaceTrack password for account clement@jonglez.space
········
Enter EDC API password for account jonglez
········


The orbit determination needs a first guess. For this, we use Two-Line Elements. Retrieving the latest TLE prior to the beginning of the orbit determination. It is important to have a "fresh" TLE, because the newer the TLE, the better the orbit estimation.

In [6]:
rawTle = st.tle(norad_cat_id=noradId, epoch='<{}'.format(startDate), orderby='epoch desc', limit=1, format='tle')
tleLine1 = rawTle.split('\n')[0]
tleLine2 = rawTle.split('\n')[1]
print(tleLine1)
print(tleLine2)

1 42829U 17042E   18194.87572169 +.00000240 +00000-0 +29796-4 0  9990
2 42829 097.5943 096.2213 0015726 108.7371 251.5556 14.90835887054309
